# Manipulate data
In this notebook the reference data and the user data are manipulated

In [1]:
from countryguess.data import Dataset, TestDataset
from countryguess.plotting import plot_sample, plot_ref_sample

## Adjust reference data
I used the following dataset <br>https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/information/<br> for the country reference shapes. The cells below modify the orginal datasets to fit the Country Guess App.

In [78]:
# Imports
import geopandas as gpd
from shapely import Polygon, MultiPolygon
from pyproj import Proj, Transformer 
from shapely.ops import transform
from pathlib import Path

In [ ]:
# Original data
ref_gdf = gpd.read_file('./world-administrative-boundaries.geojson')

### DONE: Change map projection
The projection used in the reference dataset seems to be the equirectangular projection. This leads to deformed country shapes, especially around the poles.

In [ ]:
def change_projection(polygon):
    source_proj = Proj(init='EPSG:4326')  # WGS84 geographic coordinate system
    target_proj = Proj(init='EPSG:3857')  # Web Mercator projection
    transformer = Transformer.from_proj(source_proj, target_proj)

    return transform(transformer.transform, polygon)

In [ ]:
ref_gdf['geometry'] = ref_gdf['geometry'].apply(change_projection)

In [ ]:
ref_gdf.to_file('./reference.geojson')

### DONE: Merge countries that are split
Russia and USA cross the antimeridian which causes problems. Lets translate them to fix this.

In [ ]:
for idx, row in ref_gdf.iterrows():
    geom = row["geometry"]
    xmin, _, xmax, _ = geom.bounds
    width = xmax - xmin
    if width > 20000000:
        print(row["name"], width)

In [ ]:
from shapely.affinity import translate

def merge_split_country(geom):
    xmin, _, xmax, _ = geom.bounds
    width = xmax - xmin
    polygons = []
    for poly in geom.geoms:
        _, _, maxx, _ = poly.bounds
        if maxx < 0:
            poly = translate(poly, xoff=width)
        polygons.append(poly)

    geom = polygons[0]
    for poly in polygons[1:]:
        geom = geom.union(poly)
    return geom

In [ ]:
ref_gdf.loc[ref_gdf['name']=='United States of America', 'geometry'] = merge_split_country(ref_gdf.loc[ref_gdf["name"]=='United States of America']['geometry'].item())
ref_gdf.loc[ref_gdf['name']=='Russian Federation', 'geometry'] = merge_split_country(ref_gdf.loc[ref_gdf["name"]=='Russian Federation']['geometry'].item())

In [ ]:
ref_gdf.to_file('./reference.geojson')

### Done: Filter countries with sparse area
Lets filter out the countries wich are extremly sparse, like the Federated States of Micronesia. As these are difficult to draw.

In [ ]:
def filter_coverage(geom, minimum=0.02):
    xmin, ymin, xmax, ymax = geom.bounds
    bounding_area = (ymax - ymin) * (xmax - xmin)
    coverage = geom.area / bounding_area
    return coverage > minimum

In [ ]:
ref_gdf = ref_gdf[ref_gdf['geometry'].apply(filter_coverage)].reset_index(drop=True)

In [ ]:
ref_gdf.to_file('./reference.geojson')

### DONE: Compose Polygon into MultiPolygon
Some counrty geometries are Polygons and some are MultiPolygon. Let's make everything a MultiPolygon.

In [ ]:
def to_MultiPolygon(geom):
    if isinstance(geom, Polygon):
        return MultiPolygon([geom])
    return geom

In [ ]:
ref_gdf['geometry'] = ref_gdf['geometry'].apply(to_MultiPolygon)

In [ ]:
ref_gdf.to_file('./reference.geojson')

### DONE: Filter countries with very few points
Some countries consists of very few points. These are not accurate enough for detecting drawings.

In [ ]:
def filter_few_points(geom, minimum=25):
    if isinstance(geom, Polygon):
        points = len(geom.exterior.coords)
    if isinstance(geom, MultiPolygon):
        points = sum([len(poly.exterior.coords) for poly in geom.geoms])
    return points > minimum

In [ ]:
ref_gdf = ref_gdf[ref_gdf['geometry'].apply(filter_few_points)].reset_index(drop=True)

In [ ]:
ref_gdf.to_file('./reference.geojson')

### DONE: Adjust columns
Lets rename and drop some columns.

In [ ]:
ref_gdf = ref_gdf.drop(columns=['geo_point_2d', 'iso3', 'color_code', 'iso_3166_1_alpha_2_codes', 'french_short'])
ref_gdf.rename(columns={'name': 'country_name'}, inplace=True)

In [ ]:
ref_gdf.to_file('./reference.geojson')

### DONE: Decompose single file
All the reference shapes are in a single geojson file. This limits git in version control of the data. 
<br>Let's decompose the single geojson file into one file for each reference country.

In [ ]:
def decompose_ref_geojson(file='./reference.geojson', output_dir='./data/reference/'):
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    ref_gdf = gpd.read_file(file)
    files = set()

    # Process each country
    for idx, row in ref_gdf.iterrows():
        # Create a clean filename from country name
        country_name = row['country_name']
        filename = country_name.lower().replace(' ', '_') + '.geojson'

        # Make sure no duplicate filenames 
        assert filename not in files
        files.add(filename)

        single_gdf = gpd.GeoDataFrame([row], columns=ref_gdf.columns, crs=ref_gdf.crs)
        
        # Save to file
        single_gdf.to_file(Path(output_dir) / filename, driver="GeoJSON")

In [ ]:
decompose_ref_geojson('./reference.geojson', './data/reference/')

### DONE: Rename some countries
Some countries do not have user friendly names, lets adjust those.

In [76]:
new_country_name = {
    'U.K. of Great Britain and Northern Ireland': 'United Kingdom',
    'Falkland Islands (Malvinas)': 'Falkland Islands',
    'Iran (Islamic Republic of)': 'Iran',
    "Democratic People's Republic of Korea": 'North Korea',
    'Democratic Republic of the Congo': 'DR Congo',
    "Lao People's Democratic Republic": 'Laos',
    'Libyan Arab Jamahiriya': 'Libya',
    'Moldova, Republic of': 'Moldova',
    'Republic of Korea': 'South Korea',
    'Russian Federation': 'Russia',
    'Syrian Arab Republic': 'Syria',
    'The former Yugoslav Republic of Macedonia': 'North Macedonia',
    'United Republic of Tanzania': 'Tanzania',
    'Svalbard and Jan Mayen Islands': 'Svalbard',
    'Brunei Darussalam': 'Brunei',
    'Swaziland': 'Eswatini' #TODO
}

In [79]:
files = Path("./data/reference/").glob("*.geojson")

for file in files:
    gdf = gpd.read_file(file)
    country_name = gdf["country_name"].item()
    
    if country_name in new_country_name:
        new_name = new_country_name[country_name]
        new_filename = f"{new_name.lower().replace(' ', '_')}.geojson"

        gdf["country_name"] = new_name
        
        # Save to file
        gdf.to_file(Path("./data/reference/") / new_filename, driver="GeoJSON")

        # Remove old file
        file.unlink()

        print(f"Updated: {country_name} -> {new_name}")
        print(f"Renamed file: {file.name} -> {new_filename}")

Updated: Swaziland -> Eswatini
Renamed file: swaziland.geojson -> eswatini.geojson


### DONE: Remove some territories
The dataset includes many countries and territories. <br>
Lets focus on the better known countries and territories.

In [ ]:
remove_countries = {"Aksai Chin", "Arunachal Pradesh", "Hala'ib Triangle", "Ilemi Triangle", "Jammu-Kashmir", "Kuril Islands"}

In [ ]:
files = Path("./data/reference/").glob("*.geojson")

for file in files:
    gdf = gpd.read_file(file)
    country_name = gdf["country_name"].item()
    
    if country_name in remove_countries:
        # Remove country
        file.unlink()

        print(f"Removed: {country_name}")

### DONE: Remove small islands seperated from the mainland
Some countries, like Chilem, Australia and Denmark, have a small island that is seperated from the mainland by a large distane. <br>Lets remove these "outliers" polygon

In [ ]:
def rm_outlier(geom, min_length=0.8, min_area=0.05):
    if isinstance(geom, Polygon):
        return geom

    #Get current geom boundary length 
    xmin, ymin, xmax, ymax = geom.bounds
    geom_length = (xmax - xmin) + (ymax - ymin)
        
    polygons = [poly for poly in geom.geoms]
    new_polygons = []
    for idx in range(len(polygons)):
        subset = polygons.copy()
        poly = subset.pop(idx)
        new_geom = MultiPolygon(subset)
        xmin, ymin, xmax, ymax = new_geom.bounds
        length = (xmax - xmin) + (ymax - ymin)
        
        #Check if the polygon should be omitted 
        if (length / geom_length) > min_length or (poly.area / geom.area) > min_area:
            new_polygons.append(poly)
    
    return MultiPolygon(new_polygons)

In [ ]:
files = Path("./data/reference/").glob("*.geojson")

for file in files:
    gdf = gpd.read_file(file)
    country_name = gdf["country_name"].item()
    geom = gdf["geometry"].item()
    new_geom = rm_outlier(geom)

    if geom!=new_geom:
        gdf["geometry"] = new_geom

        # Update file
        file.unlink()
        if new_geom:
            print("Adjusted:", country_name)
            gdf.to_file(file, driver="GeoJSON")
        else:
            print("Removed:", country_name)

### Reference countries
These steps leave us with the following reference countries.

In [2]:
ref_data = Dataset(shape=(128, 128))

In [186]:
plot_ref_sample(ref_data)

## User drawings

In [ ]:
test_data = TestDataset(shape=(128, 128))

In [ ]:
plot_sample(test_data, idx=77)